## Εργασία 2 ##

Καλωσήρθατε στην δεύτερη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα δενδρικά μοντέλα και την αξιολόγηση μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [189]:
NAME = "Alexandros Korkos"
AEM = "3870"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων diabetes που αφορά την πρόβλεψη της πορείας που θα έχει ένας ασθενής με διαβήτη ένα χρόνο μετά (εργασία παλινδρόμησης) και αποθηκεύστε το σε μια μεταβλητή με το όνομα diabetes. (0.5 μονάδες)

In [190]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes(as_frame=True)

In [191]:
"""Τεστ ορθής ανάγνωσης του συνόλου δεδομένων"""
assert diabetes.feature_names == ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

**2** Χωρίστε το σύνολο που διαβάσατε σε υποσύνολο ελέγχου 20% και υποσύνολο εκπαίδευσης 80% με την μέθοδο train_test_split και σπόρο τυχαιότητας 42. Aποθηκεύστε το σύνολο εκπαίδευσης σε μεταβλητές X_train, y_train και το σύνολο ελέγχου σε μεταβλητές X_test, y_test. (0.5 μονάδες)

In [192]:
from sklearn.model_selection import train_test_split

RANDOM_VARIABLE = 42

X = np.array(diabetes.data)
y = np.array(diabetes.target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_VARIABLE)

In [193]:
"""Τεστ ορθού διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.02736
assert round(X_test[0][8], 5) == 0.03243

**3** Χρησιμοποιώντας τα δεδομένα εκπαίδευσης, εκπαιδεύστε δύο δένδρα παλινδρόμησης με κριτήρια διαχωρισμού τα squared_eror και poisson ([δείτε ενότητα 1.10.7.2](https://scikit-learn.org/stable/modules/tree.html#tree)) αντίστοιχα, θέτοντας τον σπόρο τυχαιότητας στην τιμή 42 και το μέγιστο βάθος στην τιμή 3. Υπολογίστε τη μετρική r2 με βάση τις προβλέψεις τους στα δεδομένα ελέγχου, τις οποίες αποθηκεύστε στις μεταβλητές y_pred1 (squared_error) και y_pred2 (poisson) αντίστοιχα.
Αποθηκεύστε τις μετρικές στις μεταβλητές r2_squared_error και r2_poisson αντίστοιχα. (1 μονάδα)

In [194]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

y_pred1 = DecisionTreeRegressor(max_depth=3, criterion="squared_error", random_state=42).fit(X_train, y_train).predict(X_test)
y_pred2 = DecisionTreeRegressor(max_depth=3, criterion="poisson", random_state=42).fit(X_train, y_train).predict(X_test)

r2_squared_error = r2_score(y_test, y_pred1)
r2_poisson = r2_score(y_test, y_pred2)

In [195]:
"""Τεστ ορθού υπολογισμού των μετρικών"""
assert round(r2_squared_error, 3) == 0.329
assert round(r2_poisson, 3) == 0.391
assert len(y_pred1) == len(y_pred2)

**4** Αναπτύξτε κώδικα με βάση την ιδιότητα [tree_](https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py) του παρακάτω μοντέλου, προκειμένου να εμφανίσετε το μονοπάτι που ακολουθήθηκε στο δένδρο για την πρόβλεψη της πρώτης περίπτωσης του συνόλου ελέγχου. Σε περίπτωση που ελέγχεται δεύτερη φορά μια μεταβλητή στο μονοπάτι, θα πρέπει να διατηρούμε τον τελευταίο και στενότερο περιορισμό. Ακολουθούν παραδείγματα ορθής λειτουργίας, στα οποία φαίνεται και η μορφή αν-τότε που πρέπει να έχει το μονοπάτι. (4 μονάδες)

In [196]:
from sklearn.tree import export_text
model = DecisionTreeRegressor(max_depth=3, random_state=42)
model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=3, random_state=42)

In [197]:
def get_path(tree, instance, feature_names):
    instance = instance.reshape(1,-1)
    feature = tree.tree_.feature
    threshold = tree.tree_.threshold
    path_dict = {}

    node_index = 0
    while True:
        feature_name = feature_names[feature[node_index]]
        if feature[node_index] != -2:  
            if instance[0, feature[node_index]] <= threshold[node_index]:
                threshold_sign = "<="
                next_node_index = tree.tree_.children_left[node_index]
            else:
                threshold_sign = ">"
                next_node_index = tree.tree_.children_right[node_index]

            path_dict.update({(feature_name, threshold_sign): 
                round(threshold[node_index], 2)})

            if next_node_index == -1:
                break

            node_index = next_node_index
        else:
            leaf_value = round(tree.tree_.value[node_index][0][0], 2)
            break
    
    path = "αν "

    for i, (key, value) in enumerate(path_dict.items()):
        if i != len(path_dict) - 1:
            path += str(key[0]) + " " + str(key[1]) + " " + str(value) + " και "
        else:
            path += str(key[0]) + " " + str(key[1]) + " " + str(value) + \
                " τότε " + str(leaf_value)
    return path

In [198]:
"""Τεστ ορθού υπολογισμού του μονοπατιού"""
assert get_path(model, X_test[0], diabetes.feature_names) == 'αν bmi <= 0.01 και s5 > 0.01 και s4 <= 0.09 τότε 159.57'
assert get_path(model, X_test[1], diabetes.feature_names) == 'αν bmi > 0.01 και bmi <= 0.07 και s6 <= 0.03 τότε 175.8'
assert get_path(model, X_test[6], diabetes.feature_names) == 'αν bmi > 0.07 και s2 > 0.02 τότε 225.75'

**5** Διαχωρίστε τα δεδομένα diabetes σε υποσύνολα εκπαίδευσης 60% (X_train, y_train), επικύρωσης 15% (X_val, y_val) και ελέγχου 25% (X_test, y_test) με σπόρο τυχαιότητας 42. (1 μονάδα)

In [199]:
RANDOM_VARIABLE = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, 
    random_state=RANDOM_VARIABLE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=.2, random_state=RANDOM_VARIABLE)


In [200]:
"""Τεστ ορθού διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == -0.08024
assert round(X_val[0][8], 5) == 0.09924
assert round(X_test[0][8], 5) == 0.03243

**6** Υλοποιήστε συνάρτηση, η οποία δοθέντος ενός δενδρικού μοντέλου παλινδρόμησης, διερευνά όλες τις ακέραιες τιμές της μεταβλητής min_samples_leaf από ένα κάτω όριο μέχρι ένα άνω όριο και επιστρέφει την βέλτιστη με βάση το mean squared error στο παραπάνω σύνολο επικύρωσης έπειτα από εκπαίδευση στο παραπάνω σύνολο εκπαίδευσης. Επιπλέον επιστρέφει και το mean squared error (στρογγυλοποιημένο σε ακέραιο) που πετυχαίνει στο σύνολο ελέγχου το μοντέλο με την βέλτιστη επίδοση στο σύνολο επικύρωσης. (2 μονάδες)

In [201]:
from sklearn.metrics import mean_squared_error

def best_min_samples_leaf(model, min, max):
    best_min_samples = None
    best_mse_test = None
    first = True
    
    for min_samples_leaf in range(min, max + 1):
        model.min_samples_leaf = min_samples_leaf
        model.fit(X_train, y_train)
        
        mse_val = mean_squared_error(y_val, model.predict(X_val))

        if first:
            best_mse_val = mse_val
            first = False
        
        if mse_val < best_mse_val:
            best_mse_val = mse_val
            best_min_samples = min_samples_leaf
            best_mse_test = mean_squared_error(y_test, model.predict(X_test)) 
    
    return (best_min_samples, round(best_mse_test))

In [202]:
"""Τεστ ορθής υλοποίησης συνάρτησης"""
model = DecisionTreeRegressor(max_depth=3, random_state=42)
assert best_min_samples_leaf(model, 1, 8) == (7, 3358)
assert best_min_samples_leaf(model, 5, 15) == (11, 3271)

**7** Υλοποιήστε συνάρτηση η οποία δοθέντος ενός μοντέλου παλινδρόμησης, ενός συνόλου δεδομένων και μίας μετρικής, το αξιολογεί με τη μέθοδο της σταυρωτής επικύρωσης "άφησε ένα εκτός". (1 μονάδα)

In [203]:
from sklearn.model_selection import LeaveOneOut
import numpy as np

def leave_one_out(model, X, y, metric):
    loo = LeaveOneOut()
    loo.get_n_splits(X)
    
    scores = []

    X = X.to_numpy()

    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        score = metric(y_test, y_pred)
        scores.append(score)
    
    return np.mean(scores)

In [204]:
"""Τεστ ορθής υλοποίησης συνάρτησης"""
model = DecisionTreeRegressor(max_leaf_nodes=6, min_samples_leaf=2, random_state=42)
assert round(leave_one_out(model, diabetes.data, diabetes.target, mean_squared_error), 0) == 3354